In [1]:
import numpy as np
import pandas as pd

In [2]:
liquor_df = pd.read_parquet('Iowa_Liquor_Sales2020-21.parquet', engine='fastparquet')
liquor_df.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,INV-31574300001,2020-11-02,2501,Hy-Vee #2 / Ames,640 Lincolnway,Ames,50010.0,None,85.0,STORY,...,76430,Oxtails Rum Punch,6,1750,5.97,8.96,6,53.76,10.50,2.77
1,INV-28409200221,2020-07-01,2500,Hy-Vee Food Store #1 / Ames,3800 W Lincoln Way,Ames,50010.0,None,85.0,STORY,...,87410,Jose Cuervo Especial Silver,6,1750,21.00,31.50,2,63.00,3.50,0.92
2,INV-28435900006,2020-07-02,4129,Cyclone Liquors,626 Lincoln Way,Ames,50010.0,POINT (-93.618911 42.022854),85.0,STORY,...,34458,Ketel One,6,1750,22.00,33.00,6,198.00,10.50,2.77
3,INV-28437200023,2020-07-02,5135,Fareway Stores #155 / Huxley,911 Hwy 69,Huxley,50124.0,None,85.0,STORY,...,43128,Bacardi Superior,6,1750,15.00,22.50,2,45.00,3.50,0.92
4,INV-28437300011,2020-07-02,5920,Kum & Go #0131 / Huxley,1706 HWY 210,Huxley,50046.0,None,85.0,STORY,...,84172,99 Bananas Mini,10,50,6.00,9.00,3,27.00,0.15,0.03


In [3]:
liquor_df.dtypes

Invoice/Item Number              object
Date                     datetime64[ns]
Store Number                      int64
Store Name                       object
Address                          object
City                             object
Zip Code                        float64
Store Location                   object
County Number                   float64
County                           object
Category                        float64
Category Name                    object
Vendor Number                   float64
Vendor Name                      object
Item Number                       int64
Item Description                 object
Pack                              int64
Bottle Volume (ml)                int64
State Bottle Cost               float64
State Bottle Retail             float64
Bottles Sold                      int64
Sale (Dollars)                  float64
Volume Sold (Liters)            float64
Volume Sold (Gallons)           float64
dtype: object

In [4]:
liquor_df.shape

(198552, 24)

In [5]:
category_sales = liquor_df.groupby(['Category Name', 'Vendor Name']).agg({'Sale (Dollars)': np.sum}).sort_values('Sale (Dollars)', ascending=False)
category_sales.head()

Sale (Dollars)
Category Name             Vendor Name                         
American Vodkas           FIFTH GENERATION INC      1684653.70
Canadian Whiskies         DIAGEO AMERICAS           1544285.81
                          Heaven Hill Brands        1484926.65
Whiskey Liqueur           SAZERAC COMPANY  INC      1360084.20
Straight Bourbon Whiskies Jim Beam Brands           1236372.87

In [6]:
category_sales.shape

(824, 1)

In [7]:
# 'Category Name' 'Item Description' 'Sale (Dollars)' 'ranking_in_category'
cols = ['Category Name', 'Item Description', 'Sale (Dollars)']
small_df = liquor_df[cols].copy()

In [8]:
small_df.head()

,Category Name,Item Description,Sale (Dollars)
0,Cocktails /RTD,Oxtails Rum Punch,53.76
1,Mixto Tequila,Jose Cuervo Especial Silver,63.00
2,Imported Vodkas,Ketel One,198.00
3,White Rum,Bacardi Superior,45.00
4,American Schnapps,99 Bananas Mini,27.00


In [9]:
small_df[small_df['Item Description'] == 'Black Velvet']

,Category Name,Item Description,Sale (Dollars)
19,Canadian Whiskies,Black Velvet,188.16
31,Canadian Whiskies,Black Velvet,94.20
43,Canadian Whiskies,Black Velvet,94.20
57,Canadian Whiskies,Black Velvet,188.16
64,Canadian Whiskies,Black Velvet,47.04
...,...,...,...
198324,Canadian Whiskies,Black Velvet,238.80
198351,Canadian Whiskies,Black Velvet,110.64
198357,Canadian Whiskies,Black Velvet,36.88
198409,Canadian Whiskies,Black Velvet,94.20


In [10]:
small_df['ranking_in_category'] = small_df.groupby(['Category Name'],
                 as_index=False)['Sale (Dollars)'].rank(ascending=False).astype(int)
# sort_values('Sale (Dollars)',ascending=False).shape
small_df.sort_values('ranking_in_category').head()

,Category Name,Item Description,Sale (Dollars),ranking_in_category
94613,Imported Dry Gins,Hendricks Gin,5293.44,1
143635,Imported Brandies,Remy Martin Louis XIII,2775.00,1
140080,Imported Flavored Vodka,Stolichnaya Razberi,1798.80,1
131337,Aged Dark Rum,Bumbu Rum,1665.00,1
127264,Imported Vodkas,Belvedere Pure Vodka,7719.00,1


In [12]:
small_df.shape

(198552, 4)

In [15]:
rankings = small_df.groupby(['Category Name', 'Item Description'],
                           as_index=False).sum()
rankings.sort_values('Sale (Dollars)', ascending=False).head()

,Category Name,Item Description,Sale (Dollars),ranking_in_category
675,American Vodkas,Titos Handmade Vodka,1663245.70,32465981
814,Canadian Whiskies,Black Velvet,1198572.91,51358839
2008,Spiced Rum,Captain Morgan Original Spiced,776557.76,10308185
839,Canadian Whiskies,Crown Royal,721902.27,16435705
2593,Tennessee Whiskies,Jack Daniels Old #7 Black Label,544763.94,3805910


In [17]:
rankings.shape

(2699, 4)

In [19]:
small_df = liquor_df[cols].copy()
small_df['ranking_in_category'] = small_df.groupby(['Category Name', 'Item Description'],
                                                  as_index=False).sum()['Sale (Dollars)'].rank(ascending=False)

In [22]:
small_df.sort_values('ranking_in_category').head()

,Category Name,Item Description,Sale (Dollars),ranking_in_category
675,American Flavored Vodka,New Amsterdam Peach,26.25,1.0
814,Imported Flavored Vodka,Ketel One Botanical Peach & Orange Blossom,37.50,2.0
2008,American Vodkas,Blue Ox Vodka,57.60,3.0
839,Straight Bourbon Whiskies,Jim Beam Apple Mini,24.30,4.0
2593,American Flavored Vodka,UV Blue Raspberry,16.49,5.0


In [23]:
small_df.shape

(198552, 4)

In [59]:
small_df = liquor_df[cols].copy()
subs = small_df.groupby(['Category Name', 'Item Description'], as_index=False).sum()
subs['rankings_in_category'] = subs.groupby('Category Name', as_index=False)['Sale (Dollars)'].rank(ascending=False)
subs = subs[subs['rankings_in_category'] <= 3]
subs.sort_values('Sale (Dollars)', ascending=False)

,Category Name,Item Description,Sale (Dollars),rankings_in_category
675,American Vodkas,Titos Handmade Vodka,1663245.70,1.0
814,Canadian Whiskies,Black Velvet,1198572.91,1.0
2008,Spiced Rum,Captain Morgan Original Spiced,776557.76,1.0
839,Canadian Whiskies,Crown Royal,721902.27,2.0
2593,Tennessee Whiskies,Jack Daniels Old #7 Black Label,544763.94,1.0
...,...,...,...,...
582,American Sloe Gins,Mr. Boston Sloe Gin,162.60,2.0
269,American Cordials & Liqueurs,Arrow Creme de Cacao Brown,81.00,2.0
584,American Vodka,SOOH Pearl Red Berry,52.50,2.0
581,American Sloe Gins,Mr Boston Sloe Gin,24.39,3.0


In [42]:
small_df.shape

(198552, 4)

In [43]:
small_df.groupby(['Category Name', 'Item Description'], as_index=False).sum().sort_values('Sale (Dollars)', ascending=False).head()

,Category Name,Item Description,Sale (Dollars),ranking_in_category
675,American Vodkas,Titos Handmade Vodka,1663245.70,9808696
814,Canadian Whiskies,Black Velvet,1198572.91,16601932
2008,Spiced Rum,Captain Morgan Original Spiced,776557.76,4980702
839,Canadian Whiskies,Crown Royal,721902.27,2771275
2593,Tennessee Whiskies,Jack Daniels Old #7 Black Label,544763.94,2095903


In [46]:
liquor_df.groupby('Sale (Dollars)').count()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Volume Sold (Liters),Volume Sold (Gallons)
Sale (Dollars),,,,,,,,,,,,,,,,,,,,,
1.34,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1.35,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
1.70,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1.88,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2.37,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18524.88,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
18533.76,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
18816.00,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [51]:
subs.sort_values('Sale (Dollars)', ascending=False)

,Category Name,Item Description,Sale (Dollars),rankings_in_category
675,American Vodkas,Titos Handmade Vodka,1663245.70,1.0
814,Canadian Whiskies,Black Velvet,1198572.91,1.0
2008,Spiced Rum,Captain Morgan Original Spiced,776557.76,1.0
839,Canadian Whiskies,Crown Royal,721902.27,2.0
2593,Tennessee Whiskies,Jack Daniels Old #7 Black Label,544763.94,1.0
...,...,...,...,...
582,American Sloe Gins,Mr. Boston Sloe Gin,162.60,2.0
269,American Cordials & Liqueurs,Arrow Creme de Cacao Brown,81.00,2.0
584,American Vodka,SOOH Pearl Red Berry,52.50,2.0
581,American Sloe Gins,Mr Boston Sloe Gin,24.39,3.0


In [52]:
task2_df = subs.sort_values('Sale (Dollars)', ascending=False)

In [53]:
iowa_population = pd.read_csv('population.csv')
iowa_population.head()

,City,Population
0,Des Moines,215636.0
1,Cedar Rapids,132301.0
2,Davenport,102169.0
3,Sioux City,82531.0
4,Iowa City,7495.0


In [57]:
liquor_df.iloc[1:3, :8]

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location
1,INV-28409200221,2020-07-01,2500,Hy-Vee Food Store #1 / Ames,3800 W Lincoln Way,Ames,50010.0,None
2,INV-28435900006,2020-07-02,4129,Cyclone Liquors,626 Lincoln Way,Ames,50010.0,POINT (-93.618911 42.022854)


In [61]:
# 'Volume Sold (Liters)'
cols = ['City', 'Sale (Dollars)', 'Volume Sold (Liters)']
small_df = liquor_df[cols].copy()
city_merge = pd.merge(small_df, iowa_population, left_on='City', right_on='City')
city_merge.head()

,City,Sale (Dollars),Volume Sold (Liters),Population
0,Ames,53.76,10.50,66023.0
1,Ames,63.00,3.50,66023.0
2,Ames,198.00,10.50,66023.0
3,Ames,13.31,0.05,66023.0
4,Ames,29.25,2.25,66023.0


In [63]:
aggregated = city_merge.groupby('City',
                   as_index=False).agg({'Sale (Dollars)': np.sum,
                                        'Volume Sold (Liters)': np.sum,
                                        'Population': np.mean})


In [64]:
aggregated['turnover_per_capita'] = aggregated['Sale (Dollars)']/aggregated['Population']
aggregated['sold_liters_per_capita'] = aggregated['Volume Sold (Liters)']/aggregated['Population']

In [65]:
aggregated.head()

,City,Sale (Dollars),Volume Sold (Liters),Population,turnover_per_capita,sold_liters_per_capita
0,Ames,1.539368e+07,937359.47,66023.0,233.156302,14.197469
1,Anamosa,1.721346e+06,122636.70,5476.0,314.343718,22.395307
2,Britt,2.417109e+05,17971.51,1893.0,127.686704,9.493666
3,Colo,8.156024e+04,5962.65,984.0,82.886423,6.059604
4,Corwith,6.477635e+04,5276.25,222.0,291.785360,23.766892


In [66]:
aggregated.sort_values('turnover_per_capita', ascending=False).head(10)

,City,Sale (Dollars),Volume Sold (Liters),Population,turnover_per_capita,sold_liters_per_capita
30,Winterset,1.523874e+06,102522.47,526.0,2897.099677,194.909639
9,Fairfield,2.058512e+06,123046.37,1029.0,2000.497949,119.578591
5,Earlham,9.881525e+04,6154.54,135.0,731.964815,45.589185
1,Anamosa,1.721346e+06,122636.70,5476.0,314.343718,22.395307
4,Corwith,6.477635e+04,5276.25,222.0,291.785360,23.766892
11,Guttenberg,3.952436e+05,27136.79,1683.0,234.844658,16.124058
0,Ames,1.539368e+07,937359.47,66023.0,233.156302,14.197469
8,Ellsworth,1.009181e+05,6477.12,510.0,197.878549,12.700235
23,Sibley,4.709217e+05,30877.87,2611.0,180.360670,11.826070
29,Webster City,1.295503e+06,85015.95,7732.0,167.550865,10.995338


In [3]:
import holidays

In [4]:
holidays.US(state='IA', years=[2020, 2021])

{datetime.date(2020, 1, 1): "New Year's Day",
 datetime.date(2020, 1, 20): 'Martin Luther King Jr. Day',
 datetime.date(2020, 2, 12): "Lincoln's Birthday",
 datetime.date(2020, 2, 17): "Washington's Birthday",
 datetime.date(2020, 5, 25): 'Memorial Day',
 datetime.date(2020, 7, 4): 'Independence Day',
 datetime.date(2020, 7, 3): 'Independence Day (Observed)',
 datetime.date(2020, 9, 7): 'Labor Day',
 datetime.date(2020, 10, 12): 'Columbus Day',
 datetime.date(2020, 11, 11): 'Veterans Day',
 datetime.date(2020, 11, 26): 'Thanksgiving',
 datetime.date(2020, 12, 25): 'Christmas Day',
 datetime.date(2021, 1, 1): "New Year's Day",
 datetime.date(2021, 12, 31): "New Year's Day (Observed)",
 datetime.date(2021, 1, 18): 'Martin Luther King Jr. Day',
 datetime.date(2021, 2, 12): "Lincoln's Birthday",
 datetime.date(2021, 2, 15): "Washington's Birthday",
 datetime.date(2021, 5, 31): 'Memorial Day',
 datetime.date(2021, 7, 4): 'Independence Day',
 datetime.date(2021, 7, 5): 'Independence Day (Obs

In [9]:
liquor_df['Date'].max().year

2021

In [13]:
holidays_df = pd.DataFrame.from_dict(holidays.US(
    state='IA', years=list(range(liquor_df['Date'].min().year, liquor_df['Date'].max().year + 1))),
                       orient='index', columns=['Holiday'])
holidays_df.index = pd.to_datetime(holidays_df.index)

In [14]:
holidays_df.index

DatetimeIndex(['2020-01-01', '2020-01-20', '2020-02-12', '2020-02-17',
               '2020-05-25', '2020-07-04', '2020-07-03', '2020-09-07',
               '2020-10-12', '2020-11-11', '2020-11-26', '2020-12-25',
               '2021-01-01', '2021-12-31', '2021-01-18', '2021-02-12',
               '2021-02-15', '2021-05-31', '2021-07-04', '2021-07-05',
               '2021-09-06', '2021-10-11', '2021-11-11', '2021-11-25',
               '2021-12-25', '2021-12-24'],
              dtype='datetime64[ns]', freq=None)

In [15]:
holidays_df.index.rename('Date', inplace=True)

In [16]:
per_day = liquor_df.groupby( 'Date').agg({'Sale (Dollars)': 'sum'})
per_day.head()

,Sale (Dollars)
Date,
2020-01-02,122807.25
2020-01-03,61217.45
2020-01-07,16623.76
2020-01-08,103861.48
2020-01-09,77649.16


In [17]:
merged_holidays = pd.merge(per_day, holidays_df, on='Date', how='left')
merged_holidays['Holiday'].fillna('No Holiday', inplace=True)

In [18]:
merged_holidays.groupby('Holiday', as_index=False).agg(
    {'Sale (Dollars)': [np.mean, 'count']}).sort_values(('Sale (Dollars)', 'mean'), ascending=False)

Holiday Sale (Dollars)      
                                         mean count
5                 Veterans Day   157116.84000     1
2           Lincoln's Birthday   116797.65000     2
1  Independence Day (Observed)   100385.01000     1
4                   No Holiday    76037.60949   353
0                 Columbus Day     8285.42000     1
3   Martin Luther King Jr. Day     2549.28000     2
6        Washington's Birthday     1588.71000     2

In [4]:
iowa_population = pd.read_csv('population.csv')
iowa_population.set_index('City', inplace=True)
iowa_population.head(4)

,Population
City,
Des Moines,215636.0
Cedar Rapids,132301.0
Davenport,102169.0
Sioux City,82531.0


In [8]:
iowa_population.loc['Davenport', 'Population']

102169.0

In [12]:
cols = ['City', 'Sale (Dollars)', 'Volume Sold (Liters)']
small_df = liquor_df[cols].copy()
aggregated = small_df.groupby('City',
               as_index=False).agg({'Sale (Dollars)': np.sum,
                                    'Volume Sold (Liters)': np.sum})
def get_population_analytics(row):
    row['Population'] = iowa_population[iowa_population['City'] == row['City']].iloc[0, 1]
    row['turnover_per_capita'] = row['Sale (Dollars)']/row['Population']
    row['sold_liters_per_capita'] = row['Volume Sold (Liters)']/row['Population']
    return row

aggregated = aggregated.apply(get_population_analytics, axis=1)
aggregated.sort_values('turnover_per_capita', ascending=False).head(10)

,City,Sale (Dollars),Volume Sold (Liters),Population,turnover_per_capita,sold_liters_per_capita
30,Winterset,1.523874e+06,102522.47,526.0,2897.099677,194.909639
9,Fairfield,2.058512e+06,123046.37,1029.0,2000.497949,119.578591
5,Earlham,9.881525e+04,6154.54,135.0,731.964815,45.589185
1,Anamosa,1.721346e+06,122636.70,5476.0,314.343718,22.395307
4,Corwith,6.477635e+04,5276.25,222.0,291.785360,23.766892
11,Guttenberg,3.952436e+05,27136.79,1683.0,234.844658,16.124058
0,Ames,1.539368e+07,937359.47,66023.0,233.156302,14.197469
8,Ellsworth,1.009181e+05,6477.12,510.0,197.878549,12.700235
23,Sibley,4.709217e+05,30877.87,2611.0,180.360670,11.826070
29,Webster City,1.295503e+06,85015.95,7732.0,167.550865,10.995338


In [6]:
aggregated.head()

,City,Sale (Dollars),Volume Sold (Liters),Population,turnover_per_capita,sold_liters_per_capita
0,Ames,1.539368e+07,937359.47,"6 66023.0 Name: Population, dtype: float64","6 233.156302 Name: Population, dtype: float64","6 14.197469 Name: Population, dtype: float64"
1,Anamosa,1.721346e+06,122636.70,"67 5476.0 Name: Population, dtype: float64","67 314.343718 Name: Population, dtype: float64","67 22.395307 Name: Population, dtype: float64"
2,Britt,2.417109e+05,17971.51,"170 1893.0 Name: Population, dtype: float64","170 127.686704 Name: Population, dtype: flo...","170 9.493666 Name: Population, dtype: float64"
3,Colo,8.156024e+04,5962.65,"294 984.0 Name: Population, dtype: float64","294 82.886423 Name: Population, dtype: float64","294 6.059604 Name: Population, dtype: float64"
4,Corwith,6.477635e+04,5276.25,"672 222.0 Name: Population, dtype: float64","672 291.78536 Name: Population, dtype: float64","672 23.766892 Name: Population, dtype: float64"


In [11]:
iowa_population[iowa_population['City'] == 'Davenport'].iloc[0, 1]

102169.0

In [9]:
def task2_sales_per_capita(df : pd.DataFrame):
    # hier Programmieren
    iowa_population = pd.read_csv('population.csv')
    cols = ['City', 'Sale (Dollars)', 'Volume Sold (Liters)']
    small_df = df[cols].copy()
    city_merge = pd.merge(small_df, iowa_population, left_on='City', right_on='City')
    aggregated = city_merge.groupby('City',
                   as_index=False).agg({'Sale (Dollars)': np.sum,
                                        'Volume Sold (Liters)': np.sum,
                                        'Population': np.mean})
    aggregated['turnover_per_capita'] = aggregated['Sale (Dollars)']/aggregated['Population']
    aggregated['sold_liters_per_capita'] = aggregated['Volume Sold (Liters)']/aggregated['Population']

    return aggregated.sort_values('turnover_per_capita', ascending=False).head(10)

In [11]:
def apply_method(df):
    iowa_population = pd.read_csv('population.csv')
    iowa_population.set_index('City', inplace=True)
    cols = ['City', 'Sale (Dollars)', 'Volume Sold (Liters)']
    small_df = df[cols].copy()
    aggregated = small_df.groupby('City',
                   as_index=False).agg({'Sale (Dollars)': np.sum,
                                        'Volume Sold (Liters)': np.sum})
    def get_population_analytics(row):
        row['Population'] = iowa_population.loc[row['City'], 'Population']
        row['turnover_per_capita'] = row['Sale (Dollars)']/row['Population']
        row['sold_liters_per_capita'] = row['Volume Sold (Liters)']/row['Population']
        return row

    aggregated = aggregated.apply(get_population_analytics, axis=1)
    
    return aggregated.sort_values('turnover_per_capita', ascending=False).head(10)

In [12]:
%%timeit
task2_sales_per_capita(liquor_df)

98.9 ms ± 4.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
apply_method(liquor_df)

117 ms ± 1.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
import time

1623672243.6096952

In [17]:
start = time.time()
task2_sales_per_capita(liquor_df)
print(time.time() - start)

0.16962170600891113


In [18]:
start = time.time()
apply_method(liquor_df)
print(time.time() - start)

0.24258708953857422


In [3]:
liquor_df.head(2)

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,INV-31574300001,2020-11-02,2501,Hy-Vee #2 / Ames,640 Lincolnway,Ames,50010.0,None,85.0,STORY,...,76430,Oxtails Rum Punch,6,1750,5.97,8.96,6,53.76,10.5,2.77
1,INV-28409200221,2020-07-01,2500,Hy-Vee Food Store #1 / Ames,3800 W Lincoln Way,Ames,50010.0,None,85.0,STORY,...,87410,Jose Cuervo Especial Silver,6,1750,21.00,31.50,2,63.00,3.5,0.92


In [4]:
liquor_df['month'] = liquor_df['Date'].dt.month

In [5]:
liquor_df.iloc[0:2, 15:]

,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),month
0,Oxtails Rum Punch,6,1750,5.97,8.96,6,53.76,10.5,2.77,11
1,Jose Cuervo Especial Silver,6,1750,21.00,31.50,2,63.00,3.5,0.92,7


In [6]:
small_df = liquor_df[liquor_df['Date'].dt.year == 2020]
small_df.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),month
0,INV-31574300001,2020-11-02,2501,Hy-Vee #2 / Ames,640 Lincolnway,Ames,50010.0,None,85.0,STORY,...,Oxtails Rum Punch,6,1750,5.97,8.96,6,53.76,10.50,2.77,11
1,INV-28409200221,2020-07-01,2500,Hy-Vee Food Store #1 / Ames,3800 W Lincoln Way,Ames,50010.0,None,85.0,STORY,...,Jose Cuervo Especial Silver,6,1750,21.00,31.50,2,63.00,3.50,0.92,7
2,INV-28435900006,2020-07-02,4129,Cyclone Liquors,626 Lincoln Way,Ames,50010.0,POINT (-93.618911 42.022854),85.0,STORY,...,Ketel One,6,1750,22.00,33.00,6,198.00,10.50,2.77,7
3,INV-28437200023,2020-07-02,5135,Fareway Stores #155 / Huxley,911 Hwy 69,Huxley,50124.0,None,85.0,STORY,...,Bacardi Superior,6,1750,15.00,22.50,2,45.00,3.50,0.92,7
4,INV-28437300011,2020-07-02,5920,Kum & Go #0131 / Huxley,1706 HWY 210,Huxley,50046.0,None,85.0,STORY,...,99 Bananas Mini,10,50,6.00,9.00,3,27.00,0.15,0.03,7


In [7]:
small_df.groupby('month').count()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
month,,,,,,,,,,,,,,,,,,,,,
1,11199,11199,11199,11199,11199,11199,11199,9683,11199,11199,...,11199,11199,11199,11199,11199,11199,11199,11199,11199,11199
2,9843,9843,9843,9843,9843,9843,9843,8543,9843,9843,...,9843,9843,9843,9843,9843,9843,9843,9843,9843,9843
3,10382,10382,10382,10382,10382,10382,10382,9242,10382,10382,...,10382,10382,10382,10382,10382,10382,10382,10382,10382,10382
4,11913,11913,11913,11913,11913,11913,11913,10470,11913,11913,...,11913,11913,11913,11913,11913,11913,11913,11913,11913,11913
5,12865,12865,12865,12865,12865,12865,12865,11470,12865,12865,...,12865,12865,12865,12865,12865,12865,12865,12865,12865,12865
6,11404,11404,11404,11404,11404,11404,11404,10186,11404,11404,...,11404,11404,11404,11404,11404,11404,11404,11404,11404,11404
7,12461,12461,12461,12461,12461,12461,12461,10950,12461,12461,...,12461,12461,12461,12461,12461,12461,12461,12461,12461,12461
8,11460,11460,11460,11460,11460,11460,11460,9974,11460,11460,...,11460,11460,11460,11460,11460,11460,11460,11460,11460,11460
9,11936,11936,11936,11936,11936,11936,11936,9161,11936,11936,...,11936,11936,11936,11936,11936,11936,11936,11936,11936,11936


In [14]:
liquor_df.iloc[0]['Sale (Dollars)']

53.76

In [16]:
for month, frame in liquor_df.groupby('month'):
    val = frame.iloc[-1]['Sale (Dollars)'] - frame.iloc[0]['Sale (Dollars)']
    print(f'diff between last and first Sale of every month. {month} {val}')

diff between last and first Sale of every month. 1 -39.900000000000006
diff between last and first Sale of every month. 2 42.769999999999996
diff between last and first Sale of every month. 3 76.38
diff between last and first Sale of every month. 4 -16.519999999999996
diff between last and first Sale of every month. 5 -198.18
diff between last and first Sale of every month. 6 -88.8
diff between last and first Sale of every month. 7 -49.41
diff between last and first Sale of every month. 8 -88.56
diff between last and first Sale of every month. 9 206.94
diff between last and first Sale of every month. 10 -9.240000000000009
diff between last and first Sale of every month. 11 -20.009999999999998
diff between last and first Sale of every month. 12 95.75999999999999
